# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "dummy_mouse_enhancers_ensembl"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

dummy_mouse_enhancers_ensembl 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2022-06-03 06:40:51.327168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 06:40:51.701886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43369 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:a3:00.0, compute capability: 8.6


In [5]:
info(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `dummy_mouse_enhancers_ensembl` has 2 classes: negative, positive.

The length of genomic intervals ranges from 331 to 4776, with average 2369.5768595041322 and median 2381.0.

Totally 1210 sequences have been found, 968 for training and 242 for testing.


,train,test
negative,484,121
positive,484,121


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 968 files belonging to 2 classes.


In [7]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

In [8]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'c', 'g']

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [10]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [11]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


2022-06-03 06:40:56.192542: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


 1/16 [>.............................] - ETA: 49s - loss: 0.7422 - binary_accuracy: 0.5625 - f1_score: 0.6842

2022-06-03 06:40:57.617399: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


16/16 [==============================] - 4s 47ms/step - loss: 0.6846 - binary_accuracy: 0.5558 - f1_score: 0.6172
Epoch 2/10
16/16 [==============================] - 1s 33ms/step - loss: 0.6150 - binary_accuracy: 0.6715 - f1_score: 0.5379
Epoch 3/10
16/16 [==============================] - 1s 31ms/step - loss: 0.6064 - binary_accuracy: 0.6911 - f1_score: 0.5079
Epoch 4/10
16/16 [==============================] - 1s 37ms/step - loss: 0.6030 - binary_accuracy: 0.6818 - f1_score: 0.5100
Epoch 5/10
16/16 [==============================] - 1s 34ms/step - loss: 0.5987 - binary_accuracy: 0.7014 - f1_score: 0.5376
Epoch 6/10
16/16 [==============================] - 0s 28ms/step - loss: 0.5935 - binary_accuracy: 0.7076 - f1_score: 0.5676
Epoch 7/10
16/16 [==============================] - 1s 29ms/step - loss: 0.5879 - binary_accuracy: 0.7211 - f1_score: 0.5460
Epoch 8/10
16/16 [==============================] - 0s 25ms/step - loss: 0.5775 - binary_accuracy: 0.7097 - f1_score: 0.5798
Epoch 9/10


## Evaluation on the test set

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 242 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

4/4 [==============================] - 0s 33ms/step - loss: 0.8968 - binary_accuracy: 0.5000 - f1_score: 0.0000e+00


[0.896836519241333, 0.5, 0.0]